In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re

sns.set(font_scale=1.5)


#Edit these accordingly
# mutdataframe is on the mutated pdbs, ref is for the reference pdb, dssp2 is an additional tsv with the PDB as the key with new values. Can use other features in the dssp2 format and inner join
mutdataframe='TPMT-PTEN.FinalValues.tsv'
refdataframe='TPMT-PTEN.RefValues.tsv'
dssp2dataframe='TPMT-PTEN.DSSP2.tsv.2'
outdataframe='TPMT-PTEN.master.tsv'


#AA properties
aaprop= pd.read_csv('/data/super/aashish/IPy/PAHpredictor2/AAprop.tsv.2',sep="\t")

dfmut = pd.read_csv(mutdataframe, sep="\t")
dfref = pd.read_csv(refdataframe, sep="\t")
dssp2=pd.read_csv(dssp2dataframe, sep="\t")

dfmut['pdbname']=dfmut.PDB.apply(lambda x: str(x).split('-')[0])
dfref['pdbname']=dfref.rPDB.apply(lambda x: str(x).split('-')[0])
dfmut['resnum']=dfmut.PDB.apply(lambda x: str(x).split('-')[2]).astype(int)

#merging dssp2 to the main mutational dataframe
dfmut=dfmut.merge(dssp2,on='PDB',how='inner')

#Setting proper types
dfmut['bur']=dfmut['bur'].apply(lambda x: str(x).split('*')[0]).astype(float)
dfref['rbur']=dfref['rbur'].apply(lambda x: str(x).split('*')[0]).astype(float)
dfmut['PW']=dfmut['PW'].apply(lambda x: str(x).split('*')[0]).astype(float)
dfref['rPW']=dfref['rPW'].apply(lambda x: str(x).split('*')[0]).astype(float)
dfmut['tsp']=dfmut['tsp'].apply(lambda x: str(x).split('*')[0]).astype(float)
dfref['rtsp']=dfref['rtsp'].apply(lambda x: str(x).split('*')[0]).astype(float)



natodrop=[ 'Consurf', 'Consurf-neigh', 'acc','racc', 'PW', 'rPW', 'bur', 'rbur', 'tsp', 'rtsp', 'RW', 'rRW', 'amb_res_tot', 'ramb_res_tot', 'amb_nres_tot', 'ramb_nres_tot', 'plopstot', 'rplotstot', 'acc2']
ambterms=['ramb_BOND', 'ramb_ANGLE', 'ramb_DIHED', 'ramb_VDWAALS', 'ramb_EEL', 'ramb_14VDWAALS', 'ramb_14EEL', 'ramb_EGB', 'ramb_ESURF', 'ramb_Gsolv', 'amb_BOND', 'amb_ANGLE', 'amb_DIHED', 'amb_VDWAALS', 'amb_EEL', 'amb_14VDWAALS', 'amb_14EEL', 'amb_EGB', 'amb_ESURF', 'amb_Gsolv']
ambresterms=['amb_res_int', 'amb_res_vdw', 'amb_res_ele', 'amb_res_pol', 'amb_res_npol', 'amb_res_tot', 'amb_nres_int', 'amb_nres_vdw', 'amb_nres_ele', 'amb_nres_pol', 'amb_nres_npol', 'amb_nres_tot', 'ramb_res_int', 'ramb_res_vdw', 'ramb_res_ele', 'ramb_res_pol', 'ramb_res_npol', 'ramb_res_tot', 'ramb_nres_int', 'ramb_nres_vdw', 'ramb_nres_ele', 'ramb_nres_pol', 'ramb_nres_npol', 'ramb_nres_tot']
natodrop=natodrop+ambterms+ambresterms

dfall=dfmut.merge(dfref, on=['pdbname','resnum'], how='left')
#pd.to_numeric(dfall[natodrop], errors='coerce')

#dfall['dbur']=dfall.apply(lambda x: float(x['bur'].split('*',1)[0]) - float(x['rbur'].split('*',1)[0]), axis=1)
#dfall['dPW']=dfall.apply(lambda x: float(x['PW'].split('*',1)[0]) - float(x['rPW'].split('*',1)[0]), axis=1)
#dfall['dtsp']=dfall.apply(lambda x: float(x['tsp'].split('*',1)[0]) - float(x['rtsp'].split('*',1)[0]), axis=1)

dfall[natodrop]=dfall[natodrop].convert_objects(convert_numeric=True)
#dfmut.merge(dfref, on=['pdbname','resnum'], how='left')[['pdbname', 'resnum', 'mutres', 'Consurf', 'PW', 'rConsurf', 'rPW', 'rtsp', 'ddG']].dropna(subset=['rPW'])
dfall=dfall.dropna(subset=natodrop)
#dfmut2.dtypes
#dfref2.dtypes

onehotSS=dfall.SS.str.get_dummies()
dfall=dfall.join(onehotSS)


dfall['dConsurf']=dfall['Consurf']
dfall['dBfactor']=dfall['Bfactor']
dfall['dConsurf-neigh']=dfall['Consurf-neigh']
dfall['dacc']=dfall['acc']-dfall['racc']
dfall['dPW']=dfall['PW']-dfall['rPW']
dfall['dbur']=dfall['bur']-dfall['rbur']
dfall['dtsp']=dfall['tsp']-dfall['rtsp']
dfall['dRW']=dfall['RW']-dfall['rRW']
dfall['dplopstot']=dfall['plopstot']-dfall['rplotstot']
dfall['damb_BOND']=dfall['amb_BOND']-dfall['ramb_BOND']
dfall['damb_ANGLE']=dfall['amb_ANGLE']-dfall['ramb_ANGLE']
dfall['dacc2']=dfall['acc2']-dfall['acc']


dfall['damb_DIHED']=dfall['amb_DIHED']-dfall['ramb_DIHED']
dfall['damb_VDWAALS']=dfall['amb_VDWAALS']-dfall['ramb_VDWAALS']
dfall['damb_EEL']=dfall['amb_EEL']-dfall['ramb_EEL']
dfall['damb_14VDWAALS']=dfall['amb_14VDWAALS']-dfall['ramb_14VDWAALS']
dfall['damb_14EEL']=dfall['amb_14EEL']-dfall['ramb_14EEL']
dfall['damb_EGB']=dfall['amb_EGB']-dfall['ramb_EGB']
dfall['damb_ESURF']=dfall['amb_ESURF']-dfall['ramb_ESURF']
dfall['damb_Gsolv']=dfall['amb_Gsolv']-dfall['ramb_Gsolv']

dfall['damb_res_int']=dfall['amb_res_int'] - dfall['ramb_res_int']
dfall['damb_res_vdw']=dfall['amb_res_vdw'] - dfall['ramb_res_vdw']
dfall['damb_res_ele']=dfall['amb_res_ele'] - dfall['ramb_res_ele']
dfall['damb_res_pol']=dfall['amb_res_pol'] - dfall['ramb_res_pol']
dfall['damb_res_npol']=dfall['amb_res_npol'] - dfall['ramb_res_npol']
dfall['damb_res_tot']=dfall['amb_res_tot'] - dfall['ramb_res_tot']
dfall['damb_nres_int']=dfall['amb_nres_int'] - dfall['ramb_nres_int']
dfall['damb_nres_vdw']=dfall['amb_nres_vdw'] - dfall['ramb_nres_vdw']
dfall['damb_nres_ele']=dfall['amb_nres_ele'] - dfall['ramb_nres_ele']
dfall['damb_nres_pol']=dfall['amb_nres_pol'] - dfall['ramb_nres_pol']
dfall['damb_nres_npol']=dfall['amb_nres_npol'] - dfall['ramb_nres_npol']
dfall['damb_nres_tot']=dfall['amb_nres_tot'] - dfall['ramb_nres_tot']


dfall['dH']=dfall['H']
dfall['dE']=dfall['E']
dfall['dBGST']=dfall.apply(lambda x: 1 if (x['E']==0 and x['H']==1) else 0, axis=1 )

dfall['dkd']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['mutres']]['KD-hydro'].iat[0] - aaprop[aaprop['Code']==row['WTres_x']]['KD-hydro'].iat[0], axis=1)
dfall['dvol']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['mutres']]['Volume'].iat[0] - aaprop[aaprop['Code']==row['WTres_x']]['Volume'].iat[0], axis=1)
dfall['dpk']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['mutres']]['pK(R)'].iat[0] - aaprop[aaprop['Code']==row['WTres_x']]['pK(R)'].iat[0], axis=1)
dfall['dsol']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['mutres']]['Solubility'].iat[0] - aaprop[aaprop['Code']==row['WTres_x']]['Solubility'].iat[0], axis=1)
dfall['dcharge']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['mutres']]['Charge'].iat[0] - aaprop[aaprop['Code']==row['WTres_x']]['Charge'].iat[0], axis=1)

dfall['rkd']=dfall.apply(lambda row: aaprop[aaprop['Code']==row['WTres_x']]['KD-hydro'].iat[0], axis=1)
dfall['ddGclass']=dfall.apply(lambda row: 0 if row['ddG'] < 0 else 1 , axis=1)

#dfall['acc'].astype(int)

#feature_cols=['dConsurf', 'dConsurf-neigh','dPW', 'dbur', 'dtsp', 'dvol', 'dkd', 'dplopstot', 'damb_res_tot', 'damb_nres_tot', 'dRW', 'damb_BOND', 'damb_ANGLE', 'damb_DIHED', 'damb_VDWAALS', 'damb_EEL', 'damb_14VDWAALS', 'damb_14EEL', 'damb_EGB', 'damb_ESURF', 'damb_Gsolv', 'damb_res_int','damb_res_vdw','damb_res_ele','damb_res_pol','damb_res_npol','damb_res_tot','damb_nres_int','damb_nres_vdw','damb_nres_ele','damb_nres_pol','damb_nres_npol','damb_nres_tot' ]
#feature_cols=['dConsurf', 'dConsurf-neigh', 'dacc', 'dPW', 'dbur', 'dtsp', 'dvol', 'dkd', 'dplopstot', 'dBfactor', 'damb_res_tot', 'damb_nres_tot']
#feature_cols=['dConsurf', 'dConsurf-neigh', 'dacc', 'dPW', 'dbur', 'dtsp', 'dvol', 'dkd', 'dplopstot', 'dRW', 'damb_BOND', 'damb_ANGLE', 'damb_DIHED', 'damb_VDWAALS', 'damb_EEL', 'damb_14VDWAALS', 'damb_14EEL', 'damb_EGB', 'damb_ESURF', 'damb_Gsolv','damb_res_int','damb_res_vdw','damb_res_ele','damb_res_pol','damb_res_npol','damb_res_tot','damb_nres_int','damb_nres_vdw','damb_nres_ele','damb_nres_pol','damb_nres_npol','damb_nres_tot']

feature_cols=[ 'acc', 'dacc2', 'dConsurf', 'dConsurf-neigh', 'dPW', 'dbur', 'dtsp', 'dvol', 'dkd', 'dplopstot', 'dRW', 'damb_BOND', 'damb_ANGLE', 'damb_DIHED', 'damb_VDWAALS', 'damb_EEL', 'damb_14VDWAALS', 'damb_14EEL', 'damb_EGB', 'damb_ESURF', 'damb_Gsolv','damb_res_int','damb_res_vdw','damb_res_ele','damb_res_pol','damb_res_npol','damb_res_tot','damb_nres_int','damb_nres_vdw','damb_nres_ele','damb_nres_pol','damb_nres_npol','damb_nres_tot']

zscore = lambda x: (x - x.mean()) / x.std()
for col in feature_cols:
    col_zscore = col + '_zscore'
    print "'" + col_zscore + "'"
    dfall[col_zscore] = dfall.groupby('pdbname')[col].transform(zscore)
    
dfall.columns.tolist()
#df.columns
#test.columns
#dfall.columns.tolist()
dfall.to_csv(outdataframe, sep="\t")

'acc_zscore'
'dacc2_zscore'
'dConsurf_zscore'
'dConsurf-neigh_zscore'
'dPW_zscore'
'dbur_zscore'
'dtsp_zscore'
'dvol_zscore'
'dkd_zscore'
'dplopstot_zscore'
'dRW_zscore'
'damb_BOND_zscore'
'damb_ANGLE_zscore'
'damb_DIHED_zscore'
'damb_VDWAALS_zscore'
'damb_EEL_zscore'
'damb_14VDWAALS_zscore'
'damb_14EEL_zscore'
'damb_EGB_zscore'
'damb_ESURF_zscore'
'damb_Gsolv_zscore'
'damb_res_int_zscore'
'damb_res_vdw_zscore'
'damb_res_ele_zscore'
'damb_res_pol_zscore'
'damb_res_npol_zscore'
'damb_res_tot_zscore'
'damb_nres_int_zscore'
'damb_nres_vdw_zscore'
'damb_nres_ele_zscore'
'damb_nres_pol_zscore'
'damb_nres_npol_zscore'
'damb_nres_tot_zscore'
